In [ ]:
%pip install -e . --quiet

# Application 1: Visualizing a particular setup

On FUMBBL there is a great guide on defensive setups. Suppose we wish to visualize the "arrowhead" setup, advised for Undead.

_The arrowhead defense is a good defense for Undead, Necromantic, or Chaos Renegades, against a highly mobile opponent. It’s similar to a ziggurat or chevron defense, but the 3-column midfielders or safeties have been moved into the 0-column to prevent runs up the gut, and also to be able to redeploy from a central position, while the 2-column strong midfielders have been pushed out to the 3-column to form a spine screen. It’s strong in the wide zone and up the center, but the 3-column midfielders are seriously exposed, and unless they have both a lot of Strength and the Stand Firm skill, this position is highly vulnerable._

The Arrowhead Defense

```{text}
7 6 5 4|3 2 1 0 1 2 3|4 5 6 7   column
- - - -|- - x x x - -|- - - -   Line
- - - -|- - - - - - -|- - - -   -1
- - x -|x - - x - - x|- x - -   -2
- x - -|- - - x - - -|- - x -   -3
```

In [ ]:
import fumbbl_replays as fb
fb.show_boardpos(rotation = 'V', crop = 'lower')

In [ ]:
roster = fb.fetch_roster("Shambling Undead")
roster

In [ ]:
my_setup = ['setup', ['Z1: g14', 'Z2: h14', 'Z3: i14', 
                      'W1: e16', 'W2: k16', 'G1: h16', 'G2: h17', 
                      'M1: c16', 'M2: m16', 'Z4: b17', 'Z5: n17']]


positions = fb.create_position(roster, my_setup)

(positions
 .query("home_away == 'teamHome'")
 .filter(['short_name', 'positionName', 'skillArrayRoster', 'learned_skills', 'skill_colors', 'boardpos'])
)

In [ ]:
fb.add_skill_to_player(positions, "W1", "Guard")
fb.add_skill_to_player(positions, "G1", "Block")

(positions
 .query("home_away == 'teamHome'")
 .filter(['short_name', 'positionName', 'skillArrayRoster', 'learned_skills', 'skill_colors', 'boardpos'])
)



In [ ]:
fb.create_plot(positions, red_team = "teamAway", orientation = 'V', crop = "lower", skill_bands = True)

for whats the play we need to display prone and stunned as well. Should include this in the notation. Maybe x and /

my_setup = ['setup', ['Z1: g14x', 'Z2: h14', 'Z3: i14', 
                      'W1: e16', 'W2: k16/', 'G1: h16/', 'G2: h17', 
                      'M1: c16', 'M2: m16', 'Z4: b17', 'Z5: n17']]

# Application 2: saving defensive setups as PNGs for use in playbooks

We want to plot all defensive setups together with match outcome from the Tilean Team Cup.
The Tilean Team Cup was an online NAF tournament held on FUMBBL from march 2023 to may 2023.

In [ ]:
import pandas as pd
import fumbbl_replays as fb

# point this to the location of the HDF5 datasets
path_to_datasets = '../fumbbl_datasets/datasets/current/'

# FUMBBL matches
target = 'df_matches.csv'
df_matches = pd.read_csv(path_to_datasets + target) 

# # subset on tilean team cup
df_matches = df_matches.query('tournament_id == 59383')


tilean_replays = df_matches['match_id'].values

tilean_replays = tilean_replays[161:]
#tilean_replays

In [ ]:
fullrun = 1

if fullrun:
    id = []
    match_ids = []
    race_defense = []
    race_offense = []

    for match_id in tilean_replays:
        match_id, replay_id, positions, receiving_team, metadata = fb.fetch_data(match_id)  # gnome 4543329 #4528210 #4542768
        plot = fb.write_plot(match_id, positions, receiving_team, metadata, refresh = True, verbose = True)
        id.append(int(replay_id))
        match_ids.append(int(match_id))
        race_defense.append(metadata[2])
        race_offense.append(metadata[3])

    df_replays = pd.DataFrame( {"matchId": match_ids,
                                "replayId": id,
                                "raceOffense": race_offense,
                                "raceDefense": race_defense})
    target = 'kickoff_pngs/df_replays'
    df_replays.to_csv(target + '.csv', index = False)
else:
    # FUMBBL matches
    target = 'kickoff_pngs/df_replays.csv'
    df_replays = pd.read_csv(target)  

In [ ]:
len(race_offense)

In [ ]:
df_replays

In [ ]:
from PIL import Image
Image.open("kickoff_pngs/wood_elf/1606445_4447434_kickoff_lower_defense.png")

In [ ]:
my_replay = fb.fetch_replay(match_id = 4447434)
rosters = fb.extract_rosters_from_replay(my_replay)

(rosters
 .query('race == "Wood Elf"')
 .filter(['short_name', 'positionName', 'playerName', 'skillArrayRoster', 'learned_skills', 'cost', 'recoveringInjury'])
)

# Application 3: Roster development in league play



In [ ]:
import pandas as pd
import fumbbl_replays as fb

pd.set_option('display.max_colwidth', None)

# We do gnomes
matches = [4542630, 4543329, 4543359, 4544140, 4545345, 4545914, 4546710, 4546875, 4546908, 4548520, 4548537, 4548898, 4549961, 4549984, 4550284]
team_id = '1177218'

# top welf too
#matches = [4543274, 4543281, 4543501, 4543513, 4543524, 4551199, 4551662, 4551696, 4551850, 4551912, 4551947, 4551983, 4552023, 4552066, 4552172]
#team_id = '1176759'

i = 0

for match_id in matches:
    my_replay = fb.fetch_replay(match_id)
    df_positions = fb.extract_rosters_from_replay(my_replay) 
    df_positions = (df_positions
                    .query("teamId == @team_id")
                    .filter(['short_name', 'positionName', 'playerName', 'skillArrayRoster', 'learned_skills', 'cost', 'recoveringInjury'])
                    )
    df_positions['match_count'] = i+1
    if i == 0:
        res = df_positions
    else:
        res = pd.concat([res, df_positions])
    i = i + 1

    



In [ ]:
obj = res.query("match_count == 15 & recoveringInjury == 'None'").agg({'cost' : sum})
obj + 200000 + 110000 # cost plus 3x rerolls and apo plus skill value increases

In [ ]:
obj = res.query("match_count == 15 & recoveringInjury == 'None'")
obj

In [ ]:
player_names = res.query("match_count == 1")['playerName'].values

(res.query('playerName in @player_names')
 .pivot(index = ['playerName', 'positionName', 'cost'], columns = 'match_count', values = 'learned_skills')
 .sort_values(by=['cost'], ascending=False)
 .fillna(value = '-')
)

# Application 4: Plotting skills on board positions (digital loom bands)



In [ ]:
%pip install -e . --quiet

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import fumbbl_replays as fb

In [ ]:
match_id, replay_id, positions, receiving_team, metadata = fb.fetch_data(match_id = 4447439)

(positions
 .query("home_away == 'teamHome'")
 .filter(['short_name', 'positionName', 'skillArrayRoster', 'learned_skills', 'skill_colors', 'boardpos'])
)


In [ ]:
fb.get_position(positions, home_away = 'teamAway')

fb.create_plot(positions, red_team = receiving_team, skill_bands = True)

Suppose we want to add a skill.

In [ ]:
positions = fb.add_skill_to_player(positions, "L6", "Block")

(positions
 .query("home_away == 'teamHome'")
 .filter(['short_name', 'positionName', 'skillArrayRoster', 'learned_skills', 'skill_colors', 'boardpos'])
)
